# Getting Started with Time-MOE

In [1]:
import os
os.chdir("/home/ubuntu/Time-MoE")

In [2]:
import os
import random

import psutil
import torch
import matplotlib.pyplot as plt

from time_moe.datasets.time_moe_dataset import TimeMoEDataset
from time_moe.models.configuration_time_moe import TimeMoeConfig
from time_moe.models.modeling_time_moe import TimeMoeForPrediction
from time_moe.datasets.time_moe_window_dataset import TimeMoEWindowDataset
from time_moe.runner import TimeMoeRunner

plt.style.use('dark_background')
# torch.set_float32_matmul_precision('high')

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-19 02:36:36.931148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766111796.950068   27019 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766111796.956187   27019 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766111796.974564   27019 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00

In [3]:
from pathlib import Path

MODEL_DIR = Path("./model")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

myparams = dict(
    input_size=1,
    hidden_size=384,
    intermediate_size=1536,
    horizon_lengths=1,
    num_hidden_layers=12,
    num_attention_heads=12,
    num_key_value_heads=None,
    hidden_act="silu",
    num_experts_per_tok=2,
    num_experts=8,
    max_position_embeddings=32768,
    initializer_range=0.02,
    rms_norm_eps=1e-6,
    use_cache=True,
    use_dense=False,
    rope_theta=10000,
    attention_dropout=0.0,
    apply_aux_loss=True,
    router_aux_loss_factor=0.02,
    tie_word_embeddings=False,
    patch=False,
    patch_len=32,
    patch_stride=32,
)

config = TimeMoeConfig(**myparams)
config.save_pretrained(MODEL_DIR)

train_config = dict(
    from_scratch=True,
    train_steps=1,
    global_batch_size=8,
    micro_batch_size=2,          
    normalization_method="zero",
    max_length=4096,
    stride=None,
    data_path="Time-300B",
    precision="fp32",
    logging_steps=1,
    dataloader_num_workers=4,
    report_to="none",          # or wandb
    torch_compile=False,       # recommended off when profiling
    output_dir="logs/time_moe",
)

# =====================
# Runner
# =====================

runner = TimeMoeRunner(
    model_path=str(MODEL_DIR),
    output_path="logs/time_moe",
    seed=9899,
)

In [4]:
runner.train_model(**train_config)

2025-12-19 02:36:53,241 - log_util.py[pid:27019;line:52:log_in_local_rank_0] - INFO: Set global_batch_size to 8
2025-12-19 02:36:53,241 - log_util.py[pid:27019;line:52:log_in_local_rank_0] - INFO: Set micro_batch_size to 2
2025-12-19 02:36:53,242 - log_util.py[pid:27019;line:52:log_in_local_rank_0] - INFO: Set gradient_accumulation_steps to 4
2025-12-19 02:36:53,242 - log_util.py[pid:27019;line:52:log_in_local_rank_0] - INFO: Set precision to fp32
2025-12-19 02:36:53,243 - log_util.py[pid:27019;line:52:log_in_local_rank_0] - INFO: Set normalization to zero
2025-12-19 02:36:53,398 - log_util.py[pid:27019;line:52:log_in_local_rank_0] - INFO: Use Eager Attention
2025-12-19 02:36:55,127 - log_util.py[pid:27019;line:52:log_in_local_rank_0] - INFO: Load model parameters from: model
2025-12-19 02:36:55,129 - log_util.py[pid:27019;line:52:log_in_local_rank_0] - INFO: {'train_steps': 1, 'global_batch_size': 8, 'micro_batch_size': 2, 'normalization_method': 'zero', 'max_length': 4096, 'stride': 

100%|██████████| 1814/1814 [00:00<00:00, 300718.05it/s]
